In [2]:
import pandas as pd
import numpy as np
from IPython.display import display

In [4]:
dataset = pd.read_csv("raw_data_Liu.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3843 entries, 0 to 3842
Data columns (total 82 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    3843 non-null   int64  
 1   Sex                   3843 non-null   object 
 2   Age                   3843 non-null   int64  
 3   Height                3843 non-null   float64
 4   Weight                3843 non-null   float64
 5   ColdSens              3843 non-null   float64
 6   ColdExp               3843 non-null   float64
 7   Workhr                3843 non-null   float64
 8   Coffeeintake          3843 non-null   float64
 9   Vote_time             3843 non-null   object 
 10  therm_sens            3843 non-null   float64
 11  therm_pref            3843 non-null   int64  
 12  location              3843 non-null   int64  
 13  mean.Temperature_60   3585 non-null   float64
 14  grad.Temperature_60   3514 non-null   float64
 15  sd.Temperature_60    

In [301]:
labels = dataset.loc[:, 'therm_pref']

# Some of the features have to be selected
truncated_dataset = pd.concat([dataset.iloc[:, 0:9], dataset[["mean.hr_5", "mean.WristT_5", "mean.AnkleT_5", "mean.PantT_5", "mean.act_5"]]], axis=1)

In [302]:
truncated_dataset

,ID,Sex,Age,Height,Weight,ColdSens,ColdExp,Workhr,Coffeeintake,mean.hr_5,mean.WristT_5,mean.AnkleT_5,mean.PantT_5,mean.act_5
0,3,Male,31,1.70,55.0,3.5,1.6,4.0,0.0,NaN,32.748333,30.640000,22.740000,NaN
1,3,Male,31,1.70,55.0,3.5,1.6,4.0,0.0,NaN,32.901667,31.205000,25.325000,7.036822
2,3,Male,31,1.70,55.0,3.5,1.6,4.0,0.0,NaN,34.530000,30.140000,25.596667,9.249484
3,3,Male,31,1.70,55.0,3.5,1.6,4.0,0.0,NaN,34.181667,29.796667,26.085000,NaN
4,3,Male,31,1.70,55.0,3.5,1.6,4.0,0.0,NaN,34.360000,30.058333,26.315000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3838,12,Male,21,1.75,61.0,3.0,0.0,1.8,0.0,NaN,30.584400,21.542000,16.721800,NaN
3839,12,Male,21,1.75,61.0,3.0,0.0,1.8,0.0,NaN,30.022800,20.991600,16.121000,NaN
3840,12,Male,21,1.75,61.0,3.0,0.0,1.8,0.0,NaN,30.035000,23.681400,16.609400,NaN
3841,12,Male,21,1.75,61.0,3.0,0.0,1.8,0.0,NaN,28.586200,22.880800,13.865000,NaN


In [212]:
import torch
from torch.nn import Sequential
truncated_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3843 entries, 0 to 3842
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sex           3843 non-null   object 
 1   Age           3843 non-null   int64  
 2   Height        3843 non-null   float64
 3   Weight        3843 non-null   float64
 4   ColdSens      3843 non-null   float64
 5   ColdExp       3843 non-null   float64
 6   Workhr        3843 non-null   float64
 7   Coffeeintake  3843 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 240.3+ KB


In [218]:
# dictionary to convert the categorical data into numerical
dict = {key:value for value, key in enumerate(truncated_dataset.loc[:, "Sex"].unique())};
truncated_dataset.loc[:, 'Sex'] = truncated_dataset.loc[:, "Sex"].map(dict)
labels = labels.map({key:value for value, key in enumerate(sorted(labels.unique()))})

# The quantity of each of the classes, it is obvious that the no-change class is the most dominant
display(labels.value_counts())

1    2631
2     635
0     577
Name: therm_pref, dtype: int64

In [264]:
from torch.nn import Sequential, Linear, ReLU
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from numpy.random import shuffle
from torch.utils.data import Dataset, DataLoader, TensorDataset
from numpy.random import permutation
from sklearn.metrics import accuracy_score

# Form the dataset as tensors
dataset, dataset_y = truncated_dataset.to_numpy().astype(np.float32), labels.to_numpy().astype(np.int64)

# shuffle the datsset
perm = permutation(range(dataset.shape[0]))
dataset_y, dataset = torch.from_numpy(dataset_y[perm]), torch.from_numpy(dataset[perm])

# Separate the train and test subdatsets and convert to Tensors
train_test_split = int(0.8 * dataset.shape[0])
train_tensor = TensorDataset(dataset[:train_test_split], dataset_y[:train_test_split])
test_x, test_y = dataset[train_test_split:], dataset_y[train_test_split:]

# Form the dataloaders for train and test datasets
batch_size = 64
train_loader = DataLoader(train_tensor, batch_size=batch_size)
test_loader = DataLoader(test_tensor, batch_size=batch_size)

In [265]:
# Construct the model
layers = [
    Linear(8, 128),
    ReLU(),
    Linear(128, 128),
    ReLU(),
    Linear(128, 3),
    ReLU()
]

model = Sequential(*layers)
optimizer = Adam(model.parameters())
loss = CrossEntropyLoss()

In [281]:
# train the model
epochs = 10

for i in range(epochs):
    model.train()
    for _, (x_train, y_train) in enumerate(train_loader):
        results = model(x_train)
        ls = loss(results, y_train)
        
        optmizer.zero_grad()
        ls.backward()
        optimizer.step()
        
    model.eval()
    results = model(test_x)
    test_ls = loss(results, test_y)
    print(f"({i+1}/{epochs}): train_loss:{0:0,f}\t test_loss:{test_ls:0.2f}\t test_acc:{accuracy_score(test_y.numpy(), )}")


TypeError: accuracy_score() missing 1 required positional argument: 'y_pred'